In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from splinter import Browser
from pprint import pprint

In [2]:
executable_path ={'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

## NASA Mars News

In [3]:
url='https://mars.nasa.gov/news/'
browser.visit(url)

In [4]:
#html object
html = browser.html

#parse html with BS
soup= BeautifulSoup(html, 'html.parser')

#Retrive latest news Title and Paragraph text.
news_title=soup.find(class_='content_title').find('a').text
news_paragraph= soup.find(class_='article_teaser_body').text

#Print Scrapped data
print(news_title)
print(news_paragraph)

NASA Invites Students to Name Mars 2020 Rover
Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.


## JPL Mars Space Images - Feature Image

In [5]:
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_url)
#Got to full image
browser.click_link_by_id('full_image')

In [7]:
browser.click_link_by_partial_text('more info')

In [8]:
#html object
html_image = browser.html

#parse html with BS
img_soup= BeautifulSoup(html_image, 'html.parser')

#Retrive latest news Title and Paragraph text.
full_image_url= img_soup.find('figure', class_='lede').a['href']
featured_image_url= 'https://www.jpl.nasa.gov'+full_image_url
featured_image_url


'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16092_hires.jpg'

## Mars Weather - Twitter

In [9]:
weather_url= 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

In [10]:
#html object
html_weather= browser.html

#parse html with BS
weather_soup = BeautifulSoup(html_weather, 'html.parser')

#Highest level container for individual tweets
weather_tweet = weather_soup.find_all('div', class_='js-tweet-text-container')

#Look for weather related tweets only
for tweet in weather_tweet:
    mars_weather= tweet.find('p', class_='TweetTextSize').text
    if 'InSight sol' in mars_weather:
        print(mars_weather)
        break   

InSight sol 265 (2019-08-25) low -99.4ºC (-146.9ºF) high -26.3ºC (-15.3ºF)
winds from the SSE at 5.3 m/s (12.0 mph) gusting to 16.1 m/s (35.9 mph)
pressure at 7.50 hPapic.twitter.com/9YLawm67zS


## Mars Facts

In [11]:
facts_url= 'https://space-facts.com/mars/'

#Use Pandas read_html to parse the url
tables= pd.read_html(facts_url)

#Choose correct table
access_table= tables[1]
#Rename column names
access_table.columns=['Facts','Value']
facts_table = access_table.set_index('Facts')
facts_table

,Value
Facts,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [12]:
#pprint(access_table)
mars_facts_html= facts_table.to_html()

## Mars Hemispheres

In [13]:
hemispheres_url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemispheres_url)

In [14]:
html_hemispheres= browser.html

hemispheres_soup = BeautifulSoup(html_hemispheres, 'html.parser')

info = hemispheres_soup.find_all('div', class_='item')

hemispehere_image_urls = []

for items in info:
    
    #title page
    title = items.find('h3').text
    
    #partial image link
    image_url =items.find('a', class_='itemLink product-item')['href']
    #getting full image link
    full_image_url='https://astrogeology.usgs.gov'+image_url
    browser.visit(full_image_url)
    size_url = browser.html
    soup = BeautifulSoup(size_url, 'html.parser')
    img_url = 'https://astrogeology.usgs.gov'+soup.find('img', class_='wide-image')['src']
    
    hemispehere_image_urls.append({'title':title, 'img_url':img_url})

hemispehere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]